In [1]:
import netrc
info = netrc.netrc()

In [2]:
from qat.lang.AQASM import Program, H, X, CNOT, RX, I, RY, RZ, CSIGN #Gates
from qat.core import Observable, Term, Batch #Hamiltonian
import numpy as np
#from qat.plugins import ScipyMinimizePlugin
import pickle

In [3]:
from qat.core.plugins import AbstractPlugin

class GaussianNoise(AbstractPlugin,):
    def __init__(self, p, ids, hamiltonian_matrix):
        self.p = p
        self.ids = ids
        self.hamiltonian_trace = np.trace(hamiltonian_matrix)/(np.shape(hamiltonian_matrix)[0])
        self.unsuccess = 0
        self.success = 0
        self.nb_pauli_strings = 0
        self.nbshots = 0

    @staticmethod
    def count_gates(in_circ):
        one_qb_gateset = ['H', 'X', 'Y', 'Z', 'RX', 'RY', 'RZ', 'I']
        two_qb_gateset = ['CNOT', 'CSIGN']
        N_1qb = sum(in_circ.count(g) for g in one_qb_gateset)
        N_2qb = sum(in_circ.count(g) for g in two_qb_gateset)
        return N_1qb, N_2qb
    
    def compile(self, batch, _):
        self.nbshots =  batch.jobs[0].nbshots
        #nb_gates = batch.jobs[0].circuit.depth({'CNOT' : 2, 'RZ' : 1, 'H' : 1}, default = 1)
        n1, n2 = self.count_gates(batch.jobs[0].circuit)
        nb_gates = n1 + 2*n2 + self.ids
        self.success = abs((1-self.p)**nb_gates)
        self.unsuccess = (1-self.success)*self.hamiltonian_trace
        return batch 
    
    def post_process(self, batch_result):
        if batch_result.results[0].value is not None:
            for result in batch_result.results:
                if self.nbshots == 0:
                    noise =  self.unsuccess
                else: 
                    noise =  np.random.normal(self.unsuccess, self.unsuccess/np.sqrt(self.nbshots))
                result.value = self.success*result.value + noise
        return batch_result

In [4]:
nqbt = 5 # number of qubits
nruns = 0 #nbshots for observable sampling

In [5]:
dep = np.arange(1, 11, 1, dtype = int)

In [6]:
#Instantiation of Hamiltoniian
heisen = Observable(nqbt)
#Generation of Heisenberg Hamiltonian
for q_reg in range(nqbt-1):
    heisen += Observable(nqbt, pauli_terms = [Term(1., typ, [q_reg,q_reg + 1]) for typ in ['XX','YY','ZZ']])

In [7]:
#exact calculation for ground state
from qat.fermion import SpinHamiltonian

heisen_class = SpinHamiltonian(nqbits=heisen.nbqbits, terms=heisen.terms)
heisen_mat = heisen_class.get_matrix()

eigvals, eigvecs = np.linalg.eigh(heisen_mat)
g_energy = eigvals[0]
g_state = eigvecs[:,0]

In [8]:
from qat.plugins import ScipyMinimizePlugin
optimizer_scipy = ScipyMinimizePlugin(method="COBYLA",
                                  tol=1e-6,
                                  options={"maxiter": 25000},
#                                  x0=np.zeros(nqbt)
                                     )

c:\Users\cqtv201\AppData\Local\anaconda3\envs\atos\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
#from qlmaas.qpus import LinAlg
from qat.qpus import get_default_qpu
#from qlmaas.qpus import MPO
#from qlmaas.qpus import NoisyQProc
#ideal processor
#qpu_ideal = LinAlg()
qpu_ideal = get_default_qpu()

In [10]:
base = 1
expo = 6

In [11]:
#infidelity
F = base*(10**(-expo))

In [12]:
def total_ids_pl():
    return(42)

In [13]:
def count_gates(in_circ):
    one_qb_gateset = ['H', 'X', 'Y', 'Z', 'RX', 'RY', 'RZ', 'I']
    two_qb_gateset = ['CNOT', 'CSIGN']
    N_1qb = sum(in_circ.count(g) for g in one_qb_gateset)
    N_2qb = sum(in_circ.count(g) for g in two_qb_gateset)
    return N_1qb, N_2qb

In [15]:
#variational gates
E_val = []
for ct in dep:
    qprog = Program()
    qbits = qprog.qalloc(nqbt)
    #variational parameters used for generating gates (permutation of [odd/even, xx/yy/zz])
    ao = [qprog.new_var(float, 'ao_%s'%i) for i in range(ct)]
    bo = [qprog.new_var(float, 'bo_%s'%i) for i in range(ct)]
    co = [qprog.new_var(float, 'co_%s'%i) for i in range(ct)]
    ae = [qprog.new_var(float, 'ae_%s'%i) for i in range(ct)]
    be = [qprog.new_var(float, 'be_%s'%i) for i in range(ct)]
    ce = [qprog.new_var(float, 'ce_%s'%i) for i in range(ct)]
    for q_index in range(nqbt):
        X(qbits[q_index])
    for q_index in range(nqbt):
        if not q_index%2 and q_index <= nqbt-1:
            H(qbits[q_index])
    for q_index in range(nqbt):
        if not q_index%2 and q_index <= nqbt-2:
            CNOT(qbits[q_index],qbits[q_index+1])
    for it in range(ct):
        for q_index in range(nqbt): #odd Rzz
            if q_index%2 and q_index <= nqbt-2:
                CNOT(qbits[q_index],qbits[q_index+1])
                RZ(ao[it-1]/2)(qbits[q_index+1])
                #I(qbits[q_index])
                CNOT(qbits[q_index],qbits[q_index+1])
        for q_index in range(nqbt): #odd Ryy
            if q_index%2 and q_index <= nqbt-2:
                RZ(np.pi/2)(qbits[q_index])
                #I(qbits[q_index])
                RZ(np.pi/2)(qbits[q_index+1])
                #I(qbits[q_index])
                H(qbits[q_index])
                H(qbits[q_index+1])
                CNOT(qbits[q_index],qbits[q_index+1])
                RZ(bo[it-1]/2)(qbits[q_index+1])
                #I(qbits[q_index])
                CNOT(qbits[q_index],qbits[q_index+1])
                H(qbits[q_index])
                H(qbits[q_index+1])
                RZ(-np.pi/2)(qbits[q_index])
                #I(qbits[q_index])
                RZ(-np.pi/2)(qbits[q_index+1])
                #I(qbits[q_index])
        for q_index in range(nqbt): #odd Rxx
            if q_index%2 and q_index <= nqbt-2:
                H(qbits[q_index])
                H(qbits[q_index+1])
                CNOT(qbits[q_index],qbits[q_index+1])
                RZ(co[it-1]/2)(qbits[q_index+1])
                #I(qbits[q_index])
                CNOT(qbits[q_index],qbits[q_index+1])
                H(qbits[q_index])
                H(qbits[q_index+1])
        for q_index in range(nqbt): #even Rzz
            if not q_index%2 and q_index <= nqbt-2:
                CNOT(qbits[q_index],qbits[q_index+1])
                RZ(ae[it-1]/2)(qbits[q_index+1])
                #I(qbits[q_index])
                CNOT(qbits[q_index],qbits[q_index+1])
        for q_index in range(nqbt): #even Ryy
            if not q_index%2 and q_index <= nqbt-2:
                RZ(np.pi/2)(qbits[q_index])
                #I(qbits[q_index])
                RZ(np.pi/2)(qbits[q_index+1])
                #I(qbits[q_index])
                H(qbits[q_index])
                H(qbits[q_index+1])
                CNOT(qbits[q_index],qbits[q_index+1])
                RZ(be[it-1]/2)(qbits[q_index+1])
                #I(qbits[q_index])
                CNOT(qbits[q_index],qbits[q_index+1])
                H(qbits[q_index])
                H(qbits[q_index+1])
                RZ(-np.pi/2)(qbits[q_index])
                #I(qbits[q_index])
                RZ(-np.pi/2)(qbits[q_index+1])
                #I(qbits[q_index])
        for q_index in range(nqbt): #even Rxx
            if not q_index%2 and q_index <= nqbt-2:
                H(qbits[q_index])
                H(qbits[q_index+1])
                CNOT(qbits[q_index],qbits[q_index+1])
                RZ(ce[it-1]/2)(qbits[q_index+1])
                #I(qbits[q_index])
                CNOT(qbits[q_index],qbits[q_index+1])
                H(qbits[q_index])
                H(qbits[q_index+1])
    circuit = qprog.to_circ()
    ids_for_noise = 3 + ct*total_ids_pl()
    n1, n2 = count_gates(circuit)
    n_gates = n1 + 2*n2 + ids_for_noise
    print(n_gates)
    


165
315
465
615
765
915
1065
1215
1365
1515


In [ ]:
    stack_noisy = optimizer_scipy | GaussianNoise(F, ids_for_noise, heisen_mat) | qpu_ideal
    #job at hand, not the observable class but instantiated job
    job = circuit.to_job(observable = heisen, nbshots = 0)
    res_noisy = stack_noisy.submit(job)
    #res_ide = res_ideal.join()
    E_val.append(res_noisy)


In [14]:
#with open('global_noisy/noisy-%s_%s.pkl'%(base, expo), 'wb') as file:
with open('global_noisy_seeds_gates_COBY_assumptions/noisy-%s_%s.pkl'%(base, expo), 'wb') as file:
    pickle.dump(E_val, file)